# Create Masternuts and Hexbins
Dieses Script erstellt unsere Hexkarte, die Grundlage für alles. Ausserdem werden OSM-Labels geladen, um jede Zelle beschriften zu können.

In [ ]:
import geopandas as gpd
import h3
from shapely.geometry import Polygon
from shapely.ops import unary_union
import pandas as pd
from pathlib import Path
from timezonefinder import TimezoneFinder
import consts
import overpy
import json
import numpy as np
from tqdm import tqdm

In [ ]:
tf:TimezoneFinder = TimezoneFinder()

## Create Masternuts - not needed anymore -> Use Hex

In [ ]:
# Import Nuts EU
gpd_nuts_eu = gpd.read_file(consts.PATH_NUTS_EU)
gpd_nuts_eu = gpd_nuts_eu[gpd_nuts_eu.LEVL_CODE == 3]

# Remove some countries
gpd_nuts_eu = gpd_nuts_eu[~gpd_nuts_eu.CNTR_CODE.isin(['TR'])]

# Import Nuts UK
gpd_nuts_uk = gpd.read_file(consts.PATH_NUTS_UK)
gpd_nuts_uk.rename(columns={
    'nuts318cd': 'NUTS_ID',
    'nuts318nm': 'NAME_LATN'
    }, inplace=True)
gpd_nuts_uk.to_crs(gpd_nuts_eu.crs, inplace=True)

# Import Bosnia. Has no NUTS
gdf_bosnia = gpd.read_file(consts.PATH_BORDERS)
gdf_bosnia = gdf_bosnia[gdf_bosnia.CNTR_ID == 'BA']
gdf_bosnia.rename(columns={'ISO3_CODE': 'NUTS_ID', 'NAME_ENGL': 'NAME_LATN'}, inplace=True)
gdf_bosnia = gdf_bosnia[['NUTS_ID', 'NAME_LATN', 'geometry']]

# Merge
gpd_nuts = pd.concat([gpd_nuts_eu, gpd_nuts_uk, gdf_bosnia], ignore_index=True)
gpd_nuts = gpd_nuts[['NUTS_ID', 'NAME_LATN', 'geometry']]

# Add timezone information to the GeoDataFrame
gpd_nuts['timezone'] = gpd_nuts.apply(
    lambda row: tf.timezone_at(lat=row.geometry.centroid.y, lng=row.geometry.centroid.x), axis=1
)

# Drop overseas
gdf_boundaries = gpd.read_file(consts.PATH_BOUNDARIES)
gpd_nuts = gpd.overlay(gpd_nuts, gdf_boundaries, how='intersection')

# Save masternuts
gpd_nuts.to_file(consts.PATH_MASTERNUTS)

## Create Hexgrid

In [ ]:
# Define the bounding box for Europe (approximate)
min_lat, min_lon = -25.0,34.0
max_lat, max_lon = 45.0,72.0

# Define the resolution for the hexagons (adjust as needed)
resolution = 4  # This resolution corresponds to hexagons with an edge length of ~50 km

# Generate hexagons within the bounding box
hexagons = []
for lat in range(int(min_lat * 100), int(max_lat * 100), 1):
    for lon in range(int(min_lon * 100), int(max_lon * 100), 1):
        hex_id = h3.latlng_to_cell(lat / 100.0, lon / 100.0, resolution)
        hexagons.append(hex_id)

# Convert hexagons to polygons
hexagon_polygons = [Polygon(h3.cell_to_boundary(h)) for h in set(hexagons)]

# Create a GeoDataFrame
gdf_hexagons = gpd.GeoDataFrame({'geometry': hexagon_polygons})

# Set the coordinate reference system (CRS) to WGS84 (EPSG:4326)
gdf_hexagons.crs = 'EPSG:4326'

## Cut Grid

In [ ]:
# Load Country data
df_eu = gpd.read_file(Path('../data/countries-europe.gpkg'))
df_eu = df_eu[~df_eu.ISO_A2_EH.isin(['RU', 'BY', 'UA', 'IS'])]
df_eu_dissolved = df_eu.dissolve()

In [ ]:
# Cut aloung outer EU border
gdf_hexagons_cut = gpd.overlay(gdf_hexagons, df_eu_dissolved, how='intersection')
gdf_hexagons_cut = gdf_hexagons_cut[['geometry']].copy()

# Cut hexagons along country borders
gdf_hexagons_cut = gpd.overlay(gdf_hexagons_cut, df_eu, how='intersection')

# Cleanup
gdf_hexagons_cut = gdf_hexagons_cut[['geometry']]

# Add Timezone
gdf_hexagons_cut['timezone'] = gdf_hexagons_cut.apply(
    lambda row: tf.timezone_at(lat=row.geometry.centroid.y, lng=row.geometry.centroid.x), axis=1
)

gdf_hexagons_cut.reset_index(drop=False, inplace=True)
gdf_hexagons_cut.rename(columns={'index':'NUTS_ID'}, inplace=True)
gdf_hexagons_cut.to_file(consts.PATH_HEXAGON)

## Download Labels from OSM
* 19 minutes loading from OSM
* x minutes saving as GPKG :D

In [ ]:
gdf_hexagons_union = gpd.read_file(consts.PATH_HEXAGON)
gdf_hexagons_union = unary_union(gdf_hexagons_cut.geometry)
gdf_hexagons_union = gpd.GeoDataFrame({'geometry': [gdf_hexagons_union]}, crs=gdf_hexagons_cut.crs)

In [ ]:
# Create a grid with NxN cells

# Get the bounds of the union of hexagons
minx, miny, maxx, maxy = gdf_hexagons_union.total_bounds

# Define the number of cells in the grid
n_cells_x = 5
n_cells_y = 5

# Calculate the size of each cell
cell_width = (maxx - minx) / n_cells_x
cell_height = (maxy - miny) / n_cells_y

# Generate the grid cells
grid_cells = []
for i in range(n_cells_x):
    for j in range(n_cells_y):
        x1 = minx + i * cell_width
        y1 = miny + j * cell_height
        x2 = x1 + cell_width
        y2 = y1 + cell_height
        grid_cells.append(Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2), (x1, y1)]))

# Create a GeoDataFrame with the grid cells
gdf_grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs=gdf_hexagons_union.crs)
gdf_grid.reset_index(drop=False, inplace=True)

In [ ]:
api = overpy.Overpass()

def get_features(place, bounds):

    x1 = bounds[1]
    y1 = bounds[0]
    x2 = bounds[3]
    y2 = bounds[2]

    # Call Overpass
    result = api.query(f"""
        (
        node["place"="{place}"]({x1},{y1},{x2},{y2});
        );
        (._;>;);
        out body;
    """)

    # Create GeoJSON Features
    features = []
    for n in result.nodes:
        features.append({
        "type": "Feature",
        "properties": n.tags,
        "geometry": {
            "type": "Point",
            "coordinates": [
                float(n.lon),
                float(n.lat)
            ]
        }
    })

    return features

geojson = {
  "type": "FeatureCollection",
  "features": []
}

# Loop every grid cell and get osm data
for i, row in gdf_grid.iterrows():
    # if row['index'] != 11:
    #     continue
    print("Cell", row['index'])

    bounds = row.geometry.bounds    
    geojson['features'] += get_features('city', bounds)
    geojson['features'] += get_features('town', bounds)
    geojson['features'] += get_features('village', bounds)
    # geojson['features'] += get_features('hamlets', bounds)

# Temp Export (its faster than reading it from memory)
json.dump(geojson, open(Path('../export/citylabels.tmp.geojson'), 'w'), ensure_ascii=False, indent = 2)

# Read as GeoDataFrame
gdf_labels_raw = gpd.read_file('../export/citylabels.tmp.geojson')
gdf_labels = gdf_labels_raw[['geometry','name:de', 'name', 'place', 'rank', 'population']]
gdf_labels.to_file('../export/citylabels.gpkg')

## Merge OSM-Labels mit Grid, add Country

In [ ]:
gdf_labels_raw = gpd.read_file('../export/citylabels.gpkg')
gdf_hexagons_cut = gpd.read_file(consts.PATH_HEXAGON)

# Add place sorting
place_sort = {
    'city': 100,
    'town': 90,
    'village': 80
}
gdf_labels_raw['place_sort'] = gdf_labels_raw['place'].apply(lambda x: place_sort[x])

In [15]:
with tqdm(total=len(gdf_hexagons_cut)) as pbar:
    for i, row in gdf_hexagons_cut.iterrows():
        pbar.update()

        points_in_hexagon = gdf_labels_raw[gdf_labels_raw.within(row.geometry)]

        if len(points_in_hexagon) > 0:

            # Add name
            points_in_hexagon = points_in_hexagon.sort_values(['place_sort', 'population'], ascending=[False, True])
            name = points_in_hexagon.iloc[0]['name:de'] if points_in_hexagon.iloc[0]['name:de'] else points_in_hexagon.iloc[0]['name']
            gdf_hexagons_cut.loc[i, 'name'] = name

        else:

            # Add Coords as name
            coords = row.geometry.centroid.coords[0]
            name = f"N{coords[1]:.3f}, E{coords[0]:.3f}"
            gdf_hexagons_cut.loc[i, 'name'] = name

        # Add country name
        point = row.geometry.centroid
        country = df_eu[df_eu.contains(point)]['NAME_DE'].values
        gdf_hexagons_cut.loc[i, 'country'] = country[0] if len(country) > 0 else 'Unknown'

# Store
gdf_hexagons_cut.to_file(consts.PATH_HEXAGON)

100%|██████████| 6838/6838 [01:42<00:00, 66.95it/s]
